# Problema 3
# a)
Busquemos el grado "óptimo" del polinomio que mejor ajusta los datos, utilizando AIC y BIC.

In [1]:
# Author: Jake VanderPlas
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import ticker
from matplotlib.patches import FancyArrow

#----------------------------------------------------------------------
# This function adjusts matplotlib settings for a uniform feel in the textbook.
# Note that with usetex=True, fonts are rendered with LaTeX.  This may
# result in an error if LaTeX is not installed on your system.  In that case,
# you can set usetex to False.
from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=8, usetex=True)


#------------------------------------------------------------
# Define our functional form
def func(x, dy=0.1):
    return np.random.normal(np.sin(x) * x, dy)

#------------------------------------------------------------
# select the (noisy) data
np.random.seed(0)
x = np.linspace(0, 3, 22)[1:-1]
dy = 0.1
y = func(x, dy)

#------------------------------------------------------------
# Select the cross-validation points
np.random.seed(1)
x_cv = 3 * np.random.random(20)
y_cv = func(x_cv)

x_fit = np.linspace(0, 3, 1000)

#------------------------------------------------------------
# Third figure: plot errors as a function of polynomial degree d
d = np.arange(0, 21)
training_err = np.zeros(d.shape)
crossval_err = np.zeros(d.shape)

fig = plt.figure(figsize=(5, 5))
for i in range(len(d)):
    p = np.polyfit(x, y, d[i])
    training_err[i] = np.sqrt(np.sum((np.polyval(p, x) - y) ** 2)
                              / len(y))
    crossval_err[i] = np.sqrt(np.sum((np.polyval(p, x_cv) - y_cv) ** 2)
                              / len(y_cv))

BIC_train = np.sqrt(len(y)) * training_err / dy + d * np.log(len(y))
BIC_crossval = np.sqrt(len(y)) * crossval_err / dy + d * np.log(len(y))

ax = fig.add_subplot(211)
ax.plot(d, crossval_err, '--k', label='cross-validation')
ax.plot(d, training_err, '-k', label='training')
ax.plot(d, 0.1 * np.ones(d.shape), ':k')

ax.set_xlim(0, 14)
ax.set_ylim(0, 0.8)

ax.set_xlabel('polynomial degree')
ax.set_ylabel('rms error')
ax.legend(loc=2)

ax = fig.add_subplot(212)
ax.plot(d, BIC_crossval, '--k', label='cross-validation')
ax.plot(d, BIC_train, '-k', label='training')

ax.set_xlim(0, 14)
ax.set_ylim(0, 100)

ax.legend(loc=2)
ax.set_xlabel('polynomial degree')
ax.set_ylabel('BIC')

plt.show()

//anaconda/lib/python2.7/site-packages/numpy/lib/polynomial.py:594: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
//anaconda/lib/python2.7/site-packages/numpy/lib/polynomial.py:594: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
//anaconda/lib/python2.7/site-packages/numpy/lib/polynomial.py:594: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)


In [ ]:
import numpy as np

n=len(t)
t1=0.4
t4=0.7
filt = (t1<t) & (t<t4)      
col1 = np.zeros(n)
for i in range(n):   
    if filt[i] == True:
        col1[i] = -1
M = col1

p=5    #Defino el grado del polinomio a ajustar

def ajustar_modelo(t,y,err,p):
    M = col1
    for i in range(p+1):
        M = np.column_stack((M, t**i))
    B = np.linalg.inv(np.matmul(M.T,M))
    C = np.matmul(M.T,y)
    param = np.matmul(B,C)

def ajustar_polinomio(t,y,sigma,p):
    # Definimos los elementos de la matriz A:
    A = np.zeros([p+1,p+1])
    # Misma cosa para b:
    b = np.zeros(p+1)
    for k in range(p+1):
        # Calculamos los elementos de b
        b[k] = np.sum((y*(t**k))/(sigma**2))
        for j in range(p+1):
            # Misma cosa para la matriz A:
            A[j,k] = np.sum(t**(k+j)/sigma**2)
    # Resolvemos el sistema lineal:
    return np.linalg.solve(A,b)

def evaluar_polinomio(a,t):
    f = 0
    for j in range(len(a)):
        f = f + a[j]*(t**j)
    return f

t,y,sigma = np.loadtxt('t3_dataset.dat',unpack=True)
# Primero calculamos el numero de datos:
N = len(t)
# Definimos K del proceso de cross-validacion:
K = 10
# Definimos el numero de puntos en cada grupo:
N_k = int(N/10.)
# Empezamos el proceso de cross-validacion. Primero,
# definimos los grados de polinomios a probar:
grados = np.arange(15)
# Creamos un vector que guardara el error de prediccion
# de cada modelo:
CV_err = np.zeros(len(grados))
# Empezamos el proceso. Primero, creamos una lista que
# guardara los indices de todos los puntos:
idx_todos = range(0,N)
# Ahora vamos iterando entre los grupos de validacion:
for k in range(K):
    # Definimos los indices en los que se encuentra el grupo
    # de validacion:
    idx_v = range(N_k*k,N_k*(k+1))
    # El grupo de entrenamiento son todos los indices que NO
    # estan en el grupo de validacion:
    idx_e = [item for item in idx_todos \
            if item not in idx_v]
    # Ahora calculamos el CV error para cada grado de polinomio:
    contador = 0
    print len(idx_e),len(idx_v)
    for grado in grados:
        # Ajustamos el polinomio al grupo de entrenamiento:
        a = ajustar_polinomio(t[idx_e],y[idx_e],sigma[idx_e],grado)
        # Evaluamos la prediccion en el grupo de validacion:
        f = evaluar_polinomio(a,t[idx_v])
        # Obtenemos el error de prediccion:
        CV_err[contador] = CV_err[contador] + np.sum(np.abs(y[idx_v]-f))
        contador += 1
        
CV_err = CV_err/np.double(N)
import matplotlib.pyplot as plt
plt.rc('text',usetex=True)
plt.rcParams.update({'font.size': 13})
plt.plot(grados,CV_err,'ro-')
plt.xlabel(r"Grados del polinomio ($p$)")
plt.ylabel(r"$\textnormal{CV}_\textnormal{err}$")
plt.show()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import norm
from scipy.stats import chisquare
from numpy import linalg
from numpy.polynomial.polynomial import polyval

from sklearn.cross_validation import KFold

tI=0.4
tIV=0.7
sigma=30.0*10**(-6)

P=5

def box_model(time,delta):
	box_flux=np.ones(time.size)
	index=np.intersect1d(np.where(time>=tI)[0],np.where(time<=tIV)[0])
	box_flux[index]=1.0-delta
	return box_flux


def CoeffMatrix(x,n):
	N=x.size
	exponents=np.arange(n)
	M=[]
	for k in range(N):
		M.append( np.power(np.ones(n)*x[k],exponents))
	return np.array(M)	



#Read Data and indentify transit invervals
time,flux,err= np.loadtxt("datos.dat",unpack=True, skiprows=1)
index_transit=np.intersect1d(np.where(time>=tI)[0],np.where(time<=tIV)[0])
index_notransit=np.union1d(np.where(time<tI)[0],np.where(time>tIV)[0])


print "P & AIC & BIC"

Delta=[]
for P in range(0,7):
	#Fit a polynomial for the regions outside the transit
	b= linalg.lstsq(CoeffMatrix(time[index_notransit],P+1),flux[index_notransit])[0]
	#Fit the depth of the transit by substracting the polynomial
	a= linalg.lstsq(CoeffMatrix(time[index_transit],1),flux[index_transit]-polyval(time[index_transit],b))[0]

	delta=-a[0]
	Delta.append(delta)
	b[0]=b[0]-1.0
	model=box_model(time,delta)+polyval(time,b)

	plt.title(str(P)+" degree poly fit")
	plt.xlabel('Time')
	plt.ylabel('Flux')
	plt.plot(time,flux,'ro', label='Data')
	plt.plot(time,model,'b', label='Box+Poly Model')
	plt.legend()
	

	LogLike= -np.log(1.0/np.sqrt(2*3.1415*sigma*sigma))*np.sum(np.square(flux-model))/(2.0*sigma*sigma)
	AIC=2.0*P-2.0*LogLike
	BIC=np.log(time.size)*P -2.0*LogLike
	
	print str(P)+" & "+str(AIC)+" & "+str(BIC)

	plt.show()


#Kfold
#Removing the depth for simplicity
flux[index_transit]+=Delta[4]
model[index_transit]+=Delta[4]


fold=5
kf = KFold(time.size, n_folds=fold)
MSE=[]
PolyParam=[]


print "P & K-fold MSE"

for P in range(0,8):
	MSE.append(0.0)
	PolyParam.append(np.zeros(P+1))
	for train_index, test_index in kf:
		time_train, time_test = time[train_index], time[test_index]
		flux_train, flux_test = flux[train_index], flux[test_index]
		
		k_param= linalg.lstsq(CoeffMatrix(time_train,P+1),flux_train)[0]
		PolyParam[P]= PolyParam[P]+ k_param/float(fold)
		set_error= np.sum(np.square(flux_test - polyval(time_test,k_param)))
		MSE[P]+=set_error/float(time.size)
	
	print str(P)+" & "+str(MSE[P])


P & AIC & BIC
0 & 4202.1473867 & 4202.1473867
1 & 3675.92134591 & 3679.62512838
2 & 3554.67371573 & 3562.08128068
3 & 3322.10945703 & 3333.22080445
4 & 3093.63720238 & 3108.45233228
5 & 3119.02780214 & 3137.54671452
6 & 3205.82786465 & 3228.0505595
P & K-fold MSE
0 & 3.02993044341e-09
1 & 4.09737660063e-09
2 & 3.02936895926e-09
3 & 3.13858492154e-09
4 & 1.54733195123e-09
5 & 7.02052471996e-09
6 & 5.77181936478e-09
7 & 7.22815033296e-08


In [6]:
import numpy as np
from sklearn.cross_validation import KFold

kf = KFold(4, n_folds=2)
print train
print test
for train, test in kf:
    print("%s %s" % (train, test))



[0 1]
[2 3]
[2 3] [0 1]
[0 1] [2 3]


In [4]:
X = np.array([[0., 0.], [1., 1.], [-1., -1.], [2., 2.]])
y = np.array([0, 1, 0, 1])
X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]